In [1]:
import re

In [2]:
from pipe import *

## SIP test

In [3]:
class SIPSegment(ComputeSegment):
    def answer(self, resp, data):
        ans = super().answer(resp, data)
        amt = float(data['sip1'].data)
        mth = float(data['sip2'].data)
        intr = float(data['sip3'].data)
        cur = float(data['sip4'].data)
        
        sip = (amt) * (intr/1200) / ((1+intr/1200) ** mth - 1)
        
        return f'your monthly SIP is {sip}'
    

In [4]:
class RiskProfileSegment(MultiChoiceSegment):
    def __init__(self, key, q, ans_lst, point_lst):
        super().__init__(key, q, ans_lst)
        self.point_lst = point_lst
    
    def answer(self, resp):
        if not self.anext:
            self.anext = self.next
        if 0 < tryint(resp) <= len(self.ans_lst):
            self.next = self.anext
            self.data = (self.ans_lst[tryint(resp) - 1], self.point_lst[tryint(resp) - 1])
            return 'ok'
        self.next = self
        return self.err   
        
    

In [5]:
class RiskProfileComputeSegment(MultiChoiceSegment, ComputeSegment):
    def __init__(self, key, q, ans_lst, point_lst):
        super().__init__(key, q, ans_lst)
        self.point_lst = point_lst
    
    def answer(self, resp, data):
        if not self.anext:
            self.anext = self.next
        if 0 < tryint(resp) <= len(self.ans_lst):
            self.next = self.anext
            self.data = (self.ans_lst[tryint(resp) - 1], self.point_lst[tryint(resp) - 1])
            
            pts = 0
            for i in range(1, 4):
                pts += int(data[f'r{i}'].data[1])
                
            return f'Your total points: {pts}'
        self.next = self
        return self.err   
        
    

In [11]:
def _test(data):
    print(data['s1'].data)
    return data['s1'].data == 'yes'

In [12]:
bot = Composite('s', 
    Segment('s1', 'do you have debt', 'ok'),
    SkipSplitter('s2', 
                Segment('s3', 'how much', 'ok'),
                _test),
    Segment('s4', 'and deposits?', 'ok'))

In [6]:
info

<class 'pipe.Composite.Composite'>: main

In [6]:
sipbot = Composite('sip', 
    Segment('sip1', 'Desired amount at end', 'ok'),
    Segment('sip2', 'Months', 'ok'),
    Segment('sip3', 'Expected rate of interest', 'ok'),
    SIPSegment('sip4', 'Current investment', ''))


In [7]:
rprof = Composite('rprof', 
                 RiskProfileSegment('r1', 'annual income?', ['<10000', '10,000 - 100,000', 
                                                            '100,000 - 1,000,000', '>1,000,000'],
                                   [1, 2, 3, 4]),
                 RiskProfileSegment('r2', 'monthly saving?', ['No saving', '<1,000', 
                                                            '1,000 - 10,000', '>10,000'],
                                   [1, 2, 3, 4]),
                 RiskProfileComputeSegment('r3', 'interest?', ['fixed deposit', 'bonds', 
                                                            'mutual fund', 'stock options'],
                                   [1, 2, 3, 4]), 
                 )

In [8]:
import re

In [9]:
from pipe import *

In [10]:
info = Composite(
            'main',
            Segment('name', 'your name?', 'ok'),
            ValidatedSegment('mobile', 'your mobile number?', 'ok', 
                             'mobile num should be 10 digits long',
                            lambda x: re.match('\d{10}', x) is not None),
            Segment('email', 'your email?', 'ok'),            
        )

In [11]:
intro = Segment('intro', '', 'Thanks for visiting.')

In [12]:
thanks = Segment('thanks', 'thanks would you like to see ad?', 'ok')

In [13]:
bot = Composite('bot',
               intro,
               SkipSplitter('info', info, decider_fn=lambda x: x['info'].data['main'].data['name'].data == ''),
               MultiChoiceSegment('path', 'Please choose your task', ['SIP', 'Risk Profile']),
                Splitter('pth', sipbot, rprof, decider_fn=lambda x: tryint(x['path'].data[0])),
               thanks)

In [62]:
bot.data['path'].data

'SIP'

In [14]:
bot.answer('hi')

decider info True


'Thanks for visiting.'

In [15]:
print(bot.question())

your name?


In [16]:
bot.answer('Niraj')

'ok'

In [17]:
print(bot.question())

your mobile number?


In [18]:
bot.answer('9876543210')

'ok'

In [19]:
print(bot.question())

your email?


In [20]:
bot.answer('asd')

'ok'

In [21]:
print(bot.question())

Please choose your task
1. SIP
2. Risk Profile


In [22]:
bot.answer('1')

decider pth 0


'ok'

In [23]:
print(bot.question())

Desired amount at end


In [24]:
bot.answer('10000000')

'ok'

In [25]:
print(bot.question())

Months


In [26]:
bot.answer('120')

'ok'

In [27]:
print(bot.question())

Expected rate of interest


In [28]:
bot.answer('12')

'ok'

In [29]:
print(bot.question())

Current investment


In [30]:
bot.answer('300000')

'your monthly SIP is 43470.94840258731'

In [31]:
print(bot.question())

thanks would you like to see ad?


In [32]:
bot.answer('no')

'ok'

In [33]:
print(bot.question())

In [34]:
bot.answer('1')

decider info False


'Thanks for visiting.'

In [35]:
print(bot.question())

Please choose your task
1. SIP
2. Risk Profile


In [36]:
bot.answer('2')

decider pth 1


'ok'

In [37]:
print(bot.question())

annual income?
1. <10000
2. 10,000 - 100,000
3. 100,000 - 1,000,000
4. >1,000,000


In [38]:
bot.answer('3')

'ok'

In [39]:
print(bot.question())

monthly saving?
1. No saving
2. <1,000
3. 1,000 - 10,000
4. >10,000


In [40]:
bot.answer('2')

'ok'

In [41]:
print(bot.question())

interest?
1. fixed deposit
2. bonds
3. mutual fund
4. stock options


In [42]:
bot.answer('3')

'Your total points: 8'

In [43]:
print(bot.question())

thanks would you like to see ad?


In [ ]:
1

## Test looping composite model

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from flowchatbot import *

In [14]:
class OrderComputeSegment(ComputeSegment):
    def answer(self, resp, data):
        print(f'compute data {data}')
        orders = get_chained_data(data, self.key, 'orders')
        if orders is None:
            orders = []
            set_chained_data(data, self.key, 'orders', val=orders)
        order = {
            'product': get_chained_data(data, 's3', 'data'),
            'quantity': get_chained_data(data, 's4', 'data'),
        }
        orders += [order]
        print(f'orders: {orders}')
        set_chained_data(data, self.key, 'data', val=resp)
        return CHAT_RET.NEXT, {'txt': self.a}

In [15]:
def decide_order(data):
    if get_chained_data(data, 'ss1', 'c2', 's5', 'data') == 'no':
        return 0
    return 1

In [16]:
bot = TextAdapter(
    Composite('c1',
              Segment('s1', '', 'welcome'),
              SkipSplitter('ss1', 
                  LoopComposite('c2',
                        Segment('s3', 'what do you want (petrol, gas, diesel)?',
                           'ok'),
                        Segment('s4', 'in what quantity?', 'ok'),
                        OrderComputeSegment('s5', 'would you like to order more?',
                                           'thanks')
                        ),
                    decider_fn=decide_order),
              Segment('s2', 'Do you want to confirm order?', 'Thanks')
         )
)
bot.r.flushall()

True

In [17]:
bot.respond('hi', 0)

composite answer current s1 data {'pos': 's1'}
composite question current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}}
skip question data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {}}
composite question current s3 data {'pos': 's3'}
adapter data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's3': {}}}}


'welcome\nwhat do you want (petrol, gas, diesel)?'

In [18]:
bot.respond('petrol', 0)

composite answer current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's3': {}}}}
skip answer data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's3', 's3': {}}}}
composite answer current s3 data {'pos': 's3', 's3': {}}
composite question current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's4', 's3': {'data': 'petrol'}}}}
skip question data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's4', 's3': {'data': 'petrol'}}}}
composite question current s4 data {'pos': 's4', 's3': {'data': 'petrol'}}
adapter data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's4', 's3': {'data': 'petrol'}, 's4': {}}}}


'ok\nin what quantity?'

In [19]:
bot.respond('2 ltr', 0)

composite answer current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's4', 's3': {'data': 'petrol'}, 's4': {}}}}
skip answer data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's4', 's3': {'data': 'petrol'}, 's4': {}}}}
composite answer current s4 data {'pos': 's4', 's3': {'data': 'petrol'}, 's4': {}}
composite question current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's5', 's3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}}}}
skip question data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's5', 's3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}}}}
composite question current s5 data {'pos': 's5', 's3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}}
adapter data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's5', 's3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {}}}}


'ok\nwould you like to order more?'

In [20]:
bot.respond('yes', 0)

composite answer current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's5', 's3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {}}}}
skip answer data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'pos': 's5', 's3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {}}}}
composite answer current s5 data {'pos': 's5', 's3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {}}
compute data {'pos': 's5', 's3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {}}
orders: [{'product': 'petrol', 'quantity': '2 ltr'}]
composite question current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}}}}
skip question data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {'orders': [{'product': 'petrol', 'quantity'

'thanks\nwhat do you want (petrol, gas, diesel)?'

In [21]:
bot.respond('gas', 0)

composite answer current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's3'}}}
skip answer data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's3'}}}
composite answer current s3 data {'s3': {'data': 'petrol'}, 's4': {'data': '2 ltr'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's3'}
composite question current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'gas'}, 's4': {'data': '2 ltr'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's4'}}}
skip question data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'gas'}, 's

'ok\nin what quantity?'

In [22]:
bot.respond('5 kg', 0)

composite answer current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'gas'}, 's4': {'data': '2 ltr'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's4'}}}
skip answer data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'gas'}, 's4': {'data': '2 ltr'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's4'}}}
composite answer current s4 data {'s3': {'data': 'gas'}, 's4': {'data': '2 ltr'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's4'}
composite question current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'gas'}, 's4': {'data': '5 kg'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's5'}}}
skip question data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'gas'}, 's4': {'data

'ok\nwould you like to order more?'

In [23]:
bot.respond('no', 0)

composite answer current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'gas'}, 's4': {'data': '5 kg'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's5'}}}
skip answer data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'gas'}, 's4': {'data': '5 kg'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's5'}}}
composite answer current s5 data {'s3': {'data': 'gas'}, 's4': {'data': '5 kg'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's5'}
compute data {'s3': {'data': 'gas'}, 's4': {'data': '5 kg'}, 's5': {'orders': [{'product': 'petrol', 'quantity': '2 ltr'}], 'data': 'yes'}, 'pos': 's5'}
orders: [{'product': 'petrol', 'quantity': '2 ltr'}, {'product': 'gas', 'quantity': '5 kg'}]
composite question current ss1 data {'pos': 'ss1', 's1': {'data': 'hi'}, 'ss1': {'pos': 'c2', 'c2': {'s3': {'data': 'g

'thanks\nDo you want to confirm order?'

In [34]:
tryint('a123')

-1

In [2]:
ComputeSegment??

In [9]:
bot.cur

<class 'pipe.Composite.Composite'>: main

In [10]:
info.cur

<class 'pipe.Segment.ValidatedSegment'>: mobile

In [14]:
bot.args

(<class 'pipe.Segment.Segment'>: intro,
 <class 'pipe.Splitter.SkipSplitter'>: info)

In [15]:
bot.args[0].next

<class 'pipe.Splitter.SkipSplitter'>: info

In [ ]:
Loo